In [1]:
import numpy as np 
import os
import pandas as pd 
from scipy.io import wavfile

import librosa
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers, activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from datetime import datetime 

from matplotlib import pyplot as plt

In [2]:
us8k_df = pd.read_pickle("us8k_df.pkl")

In [3]:
def init_model():
    model1 = Sequential()
    
    #layer-1
    model1.add(Conv2D(filters=24, kernel_size=5, input_shape=(128, 128, 1),
                      kernel_regularizer=regularizers.l2(1e-3)))
    model1.add(MaxPooling2D(pool_size=(3,3), strides=3))
    model1.add(Activation(activations.relu))
    
    #layer-2
    model1.add(Conv2D(filters=36, kernel_size=4, padding='valid', kernel_regularizer=regularizers.l2(1e-3)))
    model1.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model1.add(Activation(activations.relu))
    
    #layer-3
    model1.add(Conv2D(filters=48, kernel_size=3, padding='valid'))
    model1.add(Activation(activations.relu))
    
    model1.add(GlobalAveragePooling2D())
    
    #layer-4 (1st dense layer)
    model1.add(Dense(60, activation='relu'))
    model1.add(Dropout(0.5))
    
    #layer-5 (2nd dense layer)
    model1.add(Dense(10, activation='softmax'))

    
    # compile
    model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model1

In [4]:
model = init_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 24)      624       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 41, 41, 24)        0         
_________________________________________________________________
activation (Activation)      (None, 41, 41, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 38, 38, 36)        13860     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 19, 36)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 19, 19, 36)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 48)        1

In [5]:
def train_test_split(fold_k, data, X_dim=(128, 128, 1)):
    X_train = np.stack(data[data.fold != fold_k].melspectrogram.to_numpy())
    X_test = np.stack(data[data.fold == fold_k].melspectrogram.to_numpy())

    y_train = data[data.fold != fold_k].label.to_numpy()
    y_test = data[data.fold == fold_k].label.to_numpy()

    XX_train = X_train.reshape(X_train.shape[0], *X_dim)
    XX_test = X_test.reshape(X_test.shape[0], *X_dim)
    
    yy_train = to_categorical(y_train)
    yy_test = to_categorical(y_test)
    
    return XX_train, XX_test, yy_train, yy_test

In [6]:
def process_fold(fold_k, data, epochs=100, num_batch_size=32):
    # split the data
    X_train, X_test, y_train, y_test = train_test_split(fold_k, data)

    # init data augmention
    #train_datagen, val_datagen = init_data_aug()
    
    # fit augmentation
    #train_datagen.fit(X_train)
    #val_datagen.fit(X_train)

    # init model
    model = init_model()

    # pre-training accuracy
    score = model.evaluate(X_test, y_test, batch_size=num_batch_size, verbose=0)
    print("Pre-training accuracy: %.4f%%\n" % (100 * score[1]))
    
    # train the model
    start = datetime.now()
    history = model.fit(X_train,y_train, epochs=epochs,validation_data=(X_test,y_test),batch_size=num_batch_size)
    end = datetime.now()
    print("Training completed in time: ", end - start, '\n')
    
    return history

In [7]:
def show_results(tot_history):
    """Show accuracy and loss graphs for train and test sets."""

    for i, history in enumerate(tot_history):
        print('\n({})'.format(i+1))

        plt.figure(figsize=(15,5))

        plt.subplot(121)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.grid(linestyle='--')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['train', 'validation'], loc='upper left')

        plt.subplot(122)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.grid(linestyle='--')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['train', 'validation'], loc='upper left')
            
        plt.show()

        print('\tMax validation accuracy: %.4f %%' % (np.max(history.history['val_accuracy']) * 100))
        print('\tMin validation loss: %.5f' % np.min(history.history['val_loss']))

In [8]:
FOLD_K = 1

history1 = []

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=10)
history1.append(history)

--------------------------------------------------------------------------------

(1)

Pre-training accuracy: 12.7148%

Epoch 1/10
246/246 [==============================] - 20s 80ms/step - loss: 2.3856 - accuracy: 0.1284 - val_loss: 2.2273 - val_accuracy: 0.1798
Epoch 2/10
246/246 [==============================] - 20s 83ms/step - loss: 2.0865 - accuracy: 0.2281 - val_loss: 1.9073 - val_accuracy: 0.3116
Epoch 3/10
246/246 [==============================] - 20s 81ms/step - loss: 1.8182 - accuracy: 0.3340 - val_loss: 1.6201 - val_accuracy: 0.4502
Epoch 4/10
246/246 [==============================] - 19s 79ms/step - loss: 1.5907 - accuracy: 0.4251 - val_loss: 1.2363 - val_accuracy: 0.5097
Epoch 5/10
246/246 [==============================] - 19s 77ms/step - loss: 1.4799 - accuracy: 0.4705 - val_loss: 1.2059 - val_accuracy: 0.6163
Epoch 6/10
246/246 [==============================] - 19s 78ms/step - loss: 1.4028 - accuracy: 0.4974 - val_loss: 1.1718 - val_accuracy: 0.5407
Epoch 7/10
246/2

In [9]:
FOLD_K = 2

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=20)
history1.append(history)

--------------------------------------------------------------------------------

(2)

Pre-training accuracy: 13.5135%

Epoch 1/20
246/246 [==============================] - 20s 79ms/step - loss: 2.2161 - accuracy: 0.1810 - val_loss: 2.0861 - val_accuracy: 0.2320
Epoch 2/20
246/246 [==============================] - 19s 78ms/step - loss: 1.7950 - accuracy: 0.3389 - val_loss: 1.7313 - val_accuracy: 0.4369
Epoch 3/20
246/246 [==============================] - 20s 83ms/step - loss: 1.5271 - accuracy: 0.4475 - val_loss: 1.5172 - val_accuracy: 0.4155
Epoch 4/20
246/246 [==============================] - 19s 79ms/step - loss: 1.3574 - accuracy: 0.5261 - val_loss: 1.2841 - val_accuracy: 0.5023
Epoch 5/20
246/246 [==============================] - 20s 79ms/step - loss: 1.2446 - accuracy: 0.5662 - val_loss: 1.2709 - val_accuracy: 0.5214
Epoch 6/20
246/246 [==============================] - 20s 82ms/step - loss: 1.1462 - accuracy: 0.6128 - val_loss: 1.3169 - val_accuracy: 0.5709
Epoch 7/20
246/2

In [10]:
FOLD_K = 3

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=20)
history1.append(history)

--------------------------------------------------------------------------------

(3)

Pre-training accuracy: 12.8649%

Epoch 1/20
244/244 [==============================] - 20s 81ms/step - loss: 2.3394 - accuracy: 0.1324 - val_loss: 2.2186 - val_accuracy: 0.2076
Epoch 2/20
244/244 [==============================] - 19s 78ms/step - loss: 2.0395 - accuracy: 0.2381 - val_loss: 1.7368 - val_accuracy: 0.3686
Epoch 3/20
244/244 [==============================] - 19s 78ms/step - loss: 1.7447 - accuracy: 0.3594 - val_loss: 1.5623 - val_accuracy: 0.4151
Epoch 4/20
244/244 [==============================] - 19s 79ms/step - loss: 1.5828 - accuracy: 0.4262 - val_loss: 1.3876 - val_accuracy: 0.4659
Epoch 5/20
244/244 [==============================] - 21s 84ms/step - loss: 1.4550 - accuracy: 0.4762 - val_loss: 1.3540 - val_accuracy: 0.4627
Epoch 6/20
244/244 [==============================] - 19s 79ms/step - loss: 1.3612 - accuracy: 0.5118 - val_loss: 1.3268 - val_accuracy: 0.5481
Epoch 7/20
244/2

In [11]:
FOLD_K = 4

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=10)
history1.append(history)

--------------------------------------------------------------------------------

(4)

Pre-training accuracy: 10.1010%

Epoch 1/10
242/242 [==============================] - 20s 82ms/step - loss: 2.2517 - accuracy: 0.1819 - val_loss: 2.1105 - val_accuracy: 0.2000
Epoch 2/10
242/242 [==============================] - 19s 80ms/step - loss: 1.8926 - accuracy: 0.2980 - val_loss: 1.8152 - val_accuracy: 0.3556
Epoch 3/10
242/242 [==============================] - 20s 83ms/step - loss: 1.6546 - accuracy: 0.3968 - val_loss: 1.7290 - val_accuracy: 0.3778
Epoch 4/10
242/242 [==============================] - 20s 84ms/step - loss: 1.5162 - accuracy: 0.4486 - val_loss: 1.4709 - val_accuracy: 0.4727
Epoch 5/10
242/242 [==============================] - 19s 79ms/step - loss: 1.4236 - accuracy: 0.4897 - val_loss: 1.3139 - val_accuracy: 0.5556
Epoch 6/10
242/242 [==============================] - 19s 79ms/step - loss: 1.3141 - accuracy: 0.5324 - val_loss: 1.2822 - val_accuracy: 0.5162
Epoch 7/10
242/2

In [12]:
FOLD_K = 5

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=30)
history1.append(history)

--------------------------------------------------------------------------------

(5)

Pre-training accuracy: 10.6838%

Epoch 1/30
244/244 [==============================] - 19s 78ms/step - loss: 2.2609 - accuracy: 0.1687 - val_loss: 1.9309 - val_accuracy: 0.2094
Epoch 2/30
244/244 [==============================] - 19s 76ms/step - loss: 1.9100 - accuracy: 0.2863 - val_loss: 1.7142 - val_accuracy: 0.3718
Epoch 3/30
244/244 [==============================] - 19s 77ms/step - loss: 1.6956 - accuracy: 0.3806 - val_loss: 1.4722 - val_accuracy: 0.4370
Epoch 4/30
244/244 [==============================] - 19s 78ms/step - loss: 1.5646 - accuracy: 0.4330 - val_loss: 1.5538 - val_accuracy: 0.4380
Epoch 5/30
244/244 [==============================] - 19s 78ms/step - loss: 1.4623 - accuracy: 0.4700 - val_loss: 1.2780 - val_accuracy: 0.5075
Epoch 6/30
244/244 [==============================] - 19s 78ms/step - loss: 1.3542 - accuracy: 0.5210 - val_loss: 1.2799 - val_accuracy: 0.5459
Epoch 7/30
244/2

In [13]:
FOLD_K = 6

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=30)
history1.append(history)

--------------------------------------------------------------------------------

(6)

Pre-training accuracy: 5.5893%

Epoch 1/30
248/248 [==============================] - 20s 78ms/step - loss: 2.2365 - accuracy: 0.1588 - val_loss: 1.9805 - val_accuracy: 0.2807
Epoch 2/30
248/248 [==============================] - 20s 80ms/step - loss: 1.8864 - accuracy: 0.2980 - val_loss: 1.7456 - val_accuracy: 0.3791
Epoch 3/30
248/248 [==============================] - 20s 79ms/step - loss: 1.6592 - accuracy: 0.3826 - val_loss: 1.5047 - val_accuracy: 0.4532
Epoch 4/30
248/248 [==============================] - 19s 76ms/step - loss: 1.4833 - accuracy: 0.4685 - val_loss: 1.3374 - val_accuracy: 0.4763
Epoch 5/30
248/248 [==============================] - 19s 75ms/step - loss: 1.3562 - accuracy: 0.5076 - val_loss: 1.3489 - val_accuracy: 0.4982
Epoch 6/30
248/248 [==============================] - 19s 76ms/step - loss: 1.2467 - accuracy: 0.5471 - val_loss: 1.2643 - val_accuracy: 0.5407
Epoch 7/30
248/24

In [14]:
FOLD_K = 7

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=30)
history1.append(history)

--------------------------------------------------------------------------------

(7)

Pre-training accuracy: 11.9332%

Epoch 1/30
247/247 [==============================] - 21s 82ms/step - loss: 2.3707 - accuracy: 0.1173 - val_loss: 2.2856 - val_accuracy: 0.1348
Epoch 2/30
247/247 [==============================] - 20s 80ms/step - loss: 2.2794 - accuracy: 0.1239 - val_loss: 2.2724 - val_accuracy: 0.1301
Epoch 3/30
247/247 [==============================] - 19s 76ms/step - loss: 2.2550 - accuracy: 0.1292 - val_loss: 2.1864 - val_accuracy: 0.1969
Epoch 4/30
247/247 [==============================] - 19s 76ms/step - loss: 2.1683 - accuracy: 0.1704 - val_loss: 2.0446 - val_accuracy: 0.2458
Epoch 5/30
247/247 [==============================] - 20s 79ms/step - loss: 2.0626 - accuracy: 0.2238 - val_loss: 1.9775 - val_accuracy: 0.2482
Epoch 6/30
247/247 [==============================] - 19s 79ms/step - loss: 1.9658 - accuracy: 0.2624 - val_loss: 1.8734 - val_accuracy: 0.2673
Epoch 7/30
247/2

In [ ]:
FOLD_K = 8

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=10)
history1.append(history)

In [ ]:
FOLD_K = 9

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=10)
history1.append(history)

In [ ]:
FOLD_K = 10

print('-'*80)
print("\n({})\n".format(FOLD_K))
    
history = process_fold(FOLD_K, us8k_df, epochs=10)
history1.append(history)